# 자체 평가지표
* 가지고 있는 제출 파일과 비교

## 데이터 가져오기

In [131]:
import pandas as pd
from tqdm import tqdm

In [132]:
# 판다스 라이브러리에서 과학적 표기 사용 안하기
pd.options.display.float_format = '{:.5f}'.format

In [133]:
# sub24_pycaret12.csv 제출 점수 : 0.1032155541
before_sub = pd.read_csv('../07_결과_출력/sub24_pycaret12.csv')
present_sub = pd.read_csv('../07_결과_출력/sub29_pycaret17.csv')

## 열을 일자, 시간, 구분 열 분해

In [134]:
# 전 체출 파일 열 정리
before_sub[['일자', '시간', '구분']] = before_sub['일자|시간|구분'].str.split(' ').to_list()
before_sub = before_sub.drop('일자|시간|구분', axis=1)
before_sub = before_sub[['일자', '시간', '구분', '공급량']]
# 현재 체출 파일 열 정리
present_sub[['일자', '시간', '구분']] = present_sub['일자|시간|구분'].str.split(' ').to_list()
present_sub = present_sub.drop('일자|시간|구분', axis=1)
present_sub = present_sub[['일자', '시간', '구분', '공급량']]

### 평가 지표 함수화

In [135]:
def evaluation_index(before_sub, present_sub):
    eval_set = pd.DataFrame(['MSE','RMSE', 'MAE', 'R2','NMAE'], columns=['평가지표'])
    for division in tqdm(present_sub['구분'].unique()):
        temp_before_sub = before_sub[before_sub['구분'] == division]
        temp_present_sub = present_sub[present_sub['구분'] == division]

        # 전 제출 값 = 실제값 취급
        # 현제 제출 값 = 예측값
        y = temp_before_sub['공급량']
        pred = temp_present_sub['공급량']
        y_mean = y.mean()

        # MSE
        mse = ( ( y - pred) ** 2 ).sum() / len(pred)
            
        # RMSE
        rmse = mse ** (1/2)
        
        # MAE
        mae = ( abs( y - pred ) ).sum() / len(pred)
        
        # R2
        upper = ( (y - pred) ** 2 ).sum()
        lower = ( (y - y_mean) ** 2 ).sum()
        if lower == 0:
            fraction = 0
        else:
            fraction = upper / lower
        r2 = 1 - fraction
        
        #NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도
        nmae = ( abs( y - pred ) / y ).sum() / len(pred)
        
        # 평가지표 리스트
        one_eval_set = [mse, rmse, mae, r2, nmae]
        
        # 평가지표 셋에 담기
        eval_set[division] = one_eval_set

    # 평가지표 평균 구하기
    eval_set['평가지표_평균'] = eval_set.mean(axis=1)

    # 평가지표 평균 출력
    print(eval_set[['평가지표', '평가지표_평균']])
    
    return eval_set

In [136]:
set = evaluation_index(before_sub, present_sub)

100%|██████████| 7/7 [00:00<00:00, 86.65it/s]

   평가지표     평가지표_평균
0   MSE 12855.27267
1  RMSE    87.75098
2   MAE    67.37401
3    R2     0.95271
4  NMAE     0.03793


In [137]:
set

,평가지표,A,B,C,D,E,G,H,평가지표_평균
0,MSE,12823.15557,8210.36372,22.92648,2342.07455,10540.93628,55703.80745,343.64468,12855.27267
1,RMSE,113.23937,90.61106,4.78816,48.39498,102.66906,236.01654,18.53766,87.75098
2,MAE,85.72759,61.75866,3.72033,34.55622,72.60595,199.74442,13.50490,67.37401
3,R2,0.94563,0.95805,0.97685,0.97020,0.97166,0.87397,0.97263,0.95271
4,NMAE,0.04569,0.03827,0.02009,0.03183,0.03098,0.06692,0.03176,0.03793
